# Basic Neural Network 

without using tensorflow

### Operations

In [2]:
class Operation:
    
    def __init__(self, input_nodes):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in self.input_nodes:
            node.output_nodes.append
            
    def compute(self):
        pass

In [3]:
class Add(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x + y

In [4]:
class Multiply(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x * y

In [5]:
class MatMul(Operation):
        
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x.dot(y)